In [2]:
import timm
import torch

model_name = "resnet50"   # ví dụ
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=1000
)

model.eval()

dummy_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(
    model,
    dummy_input,
    "model.onnx",
    opset_version=13,
    input_names=["input"],
    output_names=["output"],
    do_constant_folding=True,
    dynamic_axes=None
)


In [4]:
python3 -m onnxsim model.onnx model_sim.onnx


CondaError: Run 'conda init' before 'conda activate'



In [ ]:
from rknn.api import RKNN

rknn = RKNN(verbose=True)

rknn.config(
    mean_values=[[123.675, 116.28, 103.53]],
    std_values=[[58.395, 57.12, 57.375]],
    target_platform="rk3588",
    quantized_dtype="w8a8"
)


ret = rknn.load_onnx(model="model_sim.onnx")
assert ret == 0

ret = rknn.build(
    do_quantization=True,
    dataset="dataset.txt"
)
assert ret == 0

rknn.export_rknn("model.rknn")
rknn.release()


I rknn-toolkit2 version: 2.3.2
I Loading : 100%|███████████████████████████████████████████████| 108/108 [00:00<00:00, 6927.12it/s]
D base_optimize ...
D base_optimize done.
D 
D fold_constant ...
D fold_constant done.
D 
D correct_ops ...
D correct_ops done.
D 
D fuse_ops ...
D fuse_ops results:
D     convert_global_avgpool_to_conv: remove node = ['/global_pool/pool/GlobalAveragePool'], add node = ['/global_pool/pool/GlobalAveragePool_2conv0']
D     convert_flatten_to_reshape: remove node = ['/global_pool/flatten/Flatten'], add node = ['/global_pool/flatten/Flatten_2rs']
D     convert_gemm_by_exmatmul: remove node = ['/fc/Gemm'], add node = ['/global_pool/flatten/Flatten_output_0_tp', '/global_pool/flatten/Flatten_output_0_tp_rs', '/fc/Gemm#1', 'output_mm_tp', 'output_mm_tp_rs']
D     unsqueeze_to_4d_transpose: remove node = [], add node = ['/global_pool/flatten/Flatten_output_0_rs', '/global_pool/flatten/Flatten_output_0_tp-rs']
D     bypass_two_reshape: remove node = ['/global_pool/